# โจทย์: Automate SQL ได้ไหม?


## SQLite with Python

In [1]:
# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')

# สร้างตาราง products ด้วย engine.execute
with engine.connect() as conn:
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        price REAL NOT NULL,
        category TEXT
    )
    '''))

    # สร้างตาราง orders
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        product_id INTEGER,
        quantity INTEGER,
        order_date TEXT
    )
    '''))

In [ ]:
with engine.connect() as conn:
    # เพิ่มข้อมูลในตาราง products
    conn.execute(text('''
    INSERT INTO products (name, price, category)
    VALUES ('iPhone 14', 35900, 'Electronics')
    '''))
    
    # เพิ่มข้อมูลในตาราง orders 
    conn.execute(text('''
    INSERT INTO orders (product_id, quantity, order_date)
    VALUES (1, 2, '2023-10-20')
    '''))
    
    conn.commit()

## Faker

In [41]:
from faker import Faker

# สร้าง Faker object สำหรับภาษาไทย
fake = Faker()

fake_data = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000),
    "category": fake.random_element(
        elements=("Electronics", "Clothing", "Food", "Books")
    ),
}
fake_data

{'name': 'iPad', 'price': 24611, 'category': 'Books'}

## Faker + SQLite

In [4]:
# สร้างข้อมูลปลอมด้วย Faker
fake_product = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000), 
    "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
}
fake_product

{'name': 'iPad', 'price': 18246, 'category': 'Food'}

In [5]:
# เพิ่มข้อมูลลงใน SQLite
with engine.connect() as conn:
    conn.execute(text('''
    INSERT INTO products (name, price, category)
    VALUES (:name, :price, :category)
    '''), fake_product)
    conn.commit()

## Pandas

In [6]:
import pandas as pd

table (dataframe) = index + column + data

|index|column1|column2|column3|
|---|---|---|---|
|0|data01|data02|data03|
|1|data11|data12|data13|

In [7]:
# สร้าง DataFrame จากข้อมูลพื้นฐาน
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

,name,age,city
0,สมชาย,25,กรุงเทพ
1,สมหญิง,30,เชียงใหม่
2,สมศรี,35,ภูเก็ต


In [8]:
name_series = df["name"]
name_series

0     สมชาย
1    สมหญิง
2     สมศรี
Name: name, dtype: object

In [9]:
age_series = df["age"]
age_series

0    25
1    30
2    35
Name: age, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    3 non-null      object
 1   age     3 non-null      int64 
 2   city    3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes


In [11]:
# แสดงข้อมูลเบื้องต้นของ DataFrame
df.describe()

,age
count,3.0
mean,30.0
std,5.0
min,25.0
25%,27.5
50%,30.0
75%,32.5
max,35.0


In [ ]:
# Python สามารถนำไปวิเคราะห์ข้อมูลต่อได้

## Pandas + SQLite

In [12]:
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

,name,age,city
0,สมชาย,25,กรุงเทพ
1,สมหญิง,30,เชียงใหม่
2,สมศรี,35,ภูเก็ต


In [13]:
df.to_sql('people', engine, if_exists='append', index=False)

3

## Pandas + SQLite + Faker

In [44]:
# สร้างข้อมูลปลอมด้วย Faker
products = []   
for _ in range(100):
    product = {
        "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
        "price": fake.random_int(min=100, max=50000), 
        "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
    }
    products.append(product)

# สร้าง DataFrame สำหรับสินค้า
df_products = pd.DataFrame(products)
df_products

,name,price,category
0,MacBook,37498,Clothing
1,MacBook,26595,Clothing
2,iPad,9615,Clothing
3,iPhone,12357,Books
4,iPad,28916,Food
...,...,...,...
95,AirPods,9840,Books
96,iPhone,283,Food
97,iPhone,24461,Food
98,iPhone,2249,Electronics


In [43]:
df_products.to_sql('products', engine, if_exists='append', index=False)

100

## Challege: สร้างข้อมูลลูกค้าโดยใช้ Faker และ Pandas และเพิ่มลงใน SQLite ทุก 5 วินาที

In [16]:
# faker to df
import faker
import pandas as pd
import time
import pendulum

# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()
def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': pendulum.now('Asia/Bangkok'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)
    return customers
customers = generate_customers(2)
customer_df = pd.DataFrame(customers)
customer_df

,customer_id,name,email,phone,address,segment,join_date,loyalty_points
0,da88dd3c-a1a3-4f36-85b7-aac3a34d4d2f,Crystal Martin,victoriawerner@example.com,985-716-0080,"48298 James Tunnel\nNew Michael, VI 16708",Retail,2024-11-30 10:14:13.660931+07:00,266
1,aa81ad4e-374c-423b-a06f-0edd0600ec6a,Eric Thompson,zdennis@example.com,001-461-685-5508x8553,"2854 Avila Flat Apt. 180\nRussellfurt, VT 46461",In-Store,2024-11-30 10:14:13.673544+07:00,70


In [17]:
# df to sqlite

# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')

# สร้างข้อมูลลูกค้า
while True:
    customers = generate_customers(5)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)

Created 5 customers at 03:14:23
Created 5 customers at 03:14:28
Created 5 customers at 03:14:33
Created 5 customers at 03:14:38
Created 5 customers at 03:14:43
Created 5 customers at 03:14:48
Created 5 customers at 03:14:53


KeyboardInterrupt: 

## Final App

In [ ]:
import faker
import pandas as pd
import time
# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')
# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()

def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': pendulum.now('Asia/Bangkok'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)
    return customers

# สร้างข้อมูลลูกค้า
while True:
    customers = generate_customers(5)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)

In [ ]:
# Python สามารถ generate data เข้า database แบบ realtime ได้แล้ว

## Congratulation!!